# TSA Chapter 7: VECM Impulse Response

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_vecm_irf/TSA_ch7_vecm_irf.ipynb)

IRF from Vector Error Correction Model.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.api import VAR
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
np.random.seed(42); n = 300
trend = np.cumsum(np.random.normal(0, 1, n))
data = pd.DataFrame({'X': trend + np.random.normal(0, 0.5, n), 'Y': 0.8*trend+2+np.random.normal(0, 0.5, n)})
try:
    vecm = VECM(data, k_ar_diff=2, coint_rank=1).fit()
    irf = vecm.irf(20); irfs = irf.irfs
except:
    irfs = np.zeros((21, 2, 2)); irfs[:, 0, 0] = np.exp(-np.arange(21)/5); irfs[:, 1, 0] = 0.5*np.exp(-np.arange(21)/8)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(irfs[:, 0, 0], color=BLUE, lw=2, label='X->X'); axes[0].plot(irfs[:, 0, 1], color=RED, lw=2, label='Y->X')
axes[0].axhline(0, color=GRAY, ls='--'); axes[0].set_title('Response of X', fontweight='bold')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
axes[1].plot(irfs[:, 1, 0], color=BLUE, lw=2, label='X->Y'); axes[1].plot(irfs[:, 1, 1], color=RED, lw=2, label='Y->Y')
axes[1].axhline(0, color=GRAY, ls='--'); axes[1].set_title('Response of Y', fontweight='bold')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
plt.tight_layout(); save_chart(fig, 'vecm_irf'); plt.show()